In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank('te') # load an blank spacy model
db = DocBin() # create a DocBin object

In [ ]:
import json

In [ ]:

f = open('/kaggle/input/new-annotation/vertopal.com-jsonformatter.json') #path to the entity1.json file 
TRAIN_DATA = json.load(f)

In [ ]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

In [ ]:
import fasttext
!python -m spacy init vectors te /kaggle/input/vector/cc.te.300.vec ml_vec #path cc.te.300.vec

In [ ]:
nlp = spacy.load('/kaggle/working/te_vec')
nlp.add_pipe('ner')

In [ ]:
nlp.to_disk('/kaggle/working/te_vec')

In [ ]:
!python -m spacy train /kaggle/working/te_vec/config.cfg --output ./ --paths.train /kaggle/working/training_data.spacy --paths.dev /kaggle/input/spacy-files/test.spacy --paths.vectors /kaggle/working/te_vec 
#change paths as per the files location in the system

In [ ]:
nlp_ner= spacy.load('/kaggle/working/model-best')
doc = nlp_ner('''Any telegu text to test the working''')
spacy.displacy.render(doc, style="ent", jupyter=True)

In [ ]:
!python -m spacy package /kaggle/working/model-best ./

In [ ]:
!zip -r te_pipeline.zip /kaggle/working/model-best